Read Scraped Data (Json)

In [126]:
import pandas as pd

# Read the file as newline-delimited JSON
df = pd.read_json('/Users/moutasemhome/Human-vs.-Synthetic-Datasets-Advancing-Niche-Model-Training-for-qa/data/forum_qas_v3_append.json', lines=True)

# Display the dataframe
df.head()


category  \
0  Uncategorized   
1  Uncategorized   
2  Uncategorized   
3  Uncategorized   
4  Uncategorized   

                                                                                                                                                                   title  \
0                                  https://discuss.pytorch.org/t/why-do-i-get-typeerror-expected-np-ndarray-got-numpy-ndarray-when-i-use-torch-from-numpy-function/37525   
1  https://discuss.pytorch.org/t/w-cudaipctypes-cpp-22-producer-process-has-been-terminated-before-all-shared-cuda-tensors-released-see-note-sharing-cuda-tensors/124445   
2                                                                                                          https://discuss.pytorch.org/t/determinism-in-inference/208033   
3                                                                        https://discuss.pytorch.org/t/updating-adam-optimizer-after-modifying-model-architecture/208101   
4                                                                                                     https://discuss.pytorch.org/t/weird-pytorch-profiler-output/208078   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         question  \
0                                                                                                                                                                                                                

We have some questions with no ansswers (empty list). Let's filter them out

In [127]:
df


category  \
0      Uncategorized   
1      Uncategorized   
2      Uncategorized   
3      Uncategorized   
4      Uncategorized   
...              ...   
30629  Uncategorized   
30630  Uncategorized   
30631  Uncategorized   
30632  Uncategorized   
30633  Uncategorized   

                                                                                                                                                                       title  \
0                                      https://discuss.pytorch.org/t/why-do-i-get-typeerror-expected-np-ndarray-got-numpy-ndarray-when-i-use-torch-from-numpy-function/37525   
1      https://discuss.pytorch.org/t/w-cudaipctypes-cpp-22-producer-process-has-been-terminated-before-all-shared-cuda-tensors-released-see-note-sharing-cuda-tensors/124445   
2                                                                                                              https://discuss.pytorch.org/t/determinism-in-inference/208033   
3                                                                            https://discuss.pytorch.org/t/updating-adam-optimizer-after-modifying-model-architecture/208101   
4                                                                                                         https://discuss.pytorch.org/t/weird-pytorch-profiler-output/208078   
...                                                                                                                                                                      ...   
30629                                                                                                   https://discuss.pytorch.org/t/how-to-check-if-torch-uses-cudnn/21933   
30630                                                                                 https://discuss.pytorch.org/t/how-to-work-with-3d-torch-in-a-linear-forward-pass/32481   
30631                                                                                                         https://discuss.pytorch.org/t/nan-values-after-optim-step/9975   
30632                                                                              https://discuss.pytorch.org/t/difference-between-nn-upsample-and-nn-convtranspose2d/32653   
30633                                                                                                                https://discuss.pytorch.org/t/loss-function-error/33065   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

Let's add a column that caluclates the total comments number accross all threads

In [110]:
df = df.copy()  # Create a deep copy of the DataFrame
df['list_length'] = df['all_answers'].apply(len)


In [111]:
print(f'We got {df["list_length"].sum()} total answers')

We got 76572 total answers


Drop duplicates if any

In [112]:
df = df.drop_duplicates(subset=['title', 'question'])
df

category  \
0      Uncategorized   
1      Uncategorized   
3      Uncategorized   
5      Uncategorized   
7      Uncategorized   
...              ...   
27587        complex   
27588        complex   
27592        complex   
27593        complex   
27594        complex   

                                                                                                                                       title  \
0      [W CudaIPCTypes.cpp:22] Producer process has been terminated before all shared CUDA tensors released. See Note [Sharing CUDA tensors]   
1                                                                                                                   Determinism in inference   
3                                                                                                              Weird Pytorch profiler output   
5                                                                                                            Create a view using subsampling   
7                                                                                                                       Torch uses wrong GPU   
...                                                                                                                                      ...   
27587                                                                                                          Complex matrix multiplication   
27588                                                                                                    Making Infinity Addition Idempotent   
27592                                                                                  How to load models on multiple gpus and forward() it?   
27593                                                                                                            How model is loaded on gpu?   
27594                                                                                          Torch Tensor from `numpy` complex not working   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

Preprocess Data (Convert to lowercase, Remove special characters and digit and Remove extra whitespaces)

In [113]:
import re

def preprocess_text(text):
    # Convert to lowercase for standardization
    text = text.lower()
    # Remove certain special characters but keep punctuation used in code
    text = re.sub(r'[^a-z0-9\s.,:;(){}\[\]@#_-]', '', text)  # Keep common symbols used in code
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply to questions (and potentially answers later)
df['cleaned_question'] = df['question'].apply(preprocess_text)

# Display the comparison between cleaned and original text
comparison_df = df[['question', 'cleaned_question']].head(1)
print(comparison_df)


                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       question  \
0  I got this warning at the end of each epoch when using multiple GPUs:\n[W CudaIPCTypes.cpp:22] Producer process has been terminated before all shared CUDA tensors released. See Note [Sharing CUDA tensors]\nBut it doesn’t seem to affect the training since the result is as good as it is.\nBut I would still like to know what the probable cause is and how to solve it  \nI

Encode Categories

In [119]:
df['category'].unique()

array(['Uncategorized', 'vision', 'projects', 'autograd', 'data',
       'reinforcement-learning', 'nlp', 'distributed', 'quantization',
       'deployment', 'PyTorch Live', 'audio', 'windows',
       'mixed-precision', 'Memory Format', 'jit', 'Mobile',
       'torch.package / torch::deploy', 'Mac OS X', 'Opacus', 'Captum',
       'tensorboard', 'ignite', 'glow', 'xla', 'Site Feedback', 'complex'],
      dtype=object)

In [120]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
categories_encoded = encoder.fit_transform(df['category'].values.reshape(-1, 1)).toarray()
categories_encoded


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Let's use Bert Tokenizer (BERT’s strong contextual understanding and performance in these types of tasks make it a suitable choice.)

https://gluebenchmark.com/leaderboard
QNLI is the most relevant since it directly evaluates question-answering capabilities.
